In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from nltk.stem.snowball import EnglishStemmer
from scipy.sparse.linalg import svds, eigs
from sklearn import datasets
import tensorflow as tf
from tensorflow.keras import layers
%matplotlib inline

In [3]:
beer = pd.read_csv("full_data.csv", index_col=0)
beer.dropna(inplace=True)

In [4]:
stemmer = EnglishStemmer()

In [5]:
analyzer = CountVectorizer(
    stop_words = 'english',
    ngram_range = (1,2)).build_analyzer()

def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

stem_vectorizer = CountVectorizer(
#     stop_words = 'english',
    analyzer=stemmed_words,
#     ngram_range = (1,3)
)
tfidf_transformer = TfidfTransformer()

In [6]:
## Vectorize the reviews
X_train_counts = stem_vectorizer.fit_transform(beer['review/text'])
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [7]:
X_train_tfidf

<37489x754033 sparse matrix of type '<class 'numpy.float64'>'
	with 4484008 stored elements in Compressed Sparse Row format>

In [ ]:
u, s, v = svds(X_train_tfidf, k = 200)

In [9]:
pd.DataFrame(u).to_csv('bigrams_100.csv')

In [10]:
u.shape

(37489, 100)

In [11]:
u = pd.read_csv('bigrams_100.csv', index_col=0)

In [12]:
full_data = pd.concat([beer, pd.DataFrame(u)], axis=1, sort=False)

In [13]:
full_data.drop('review/text', inplace=True, axis=1)

In [11]:
X = full_data.drop(['review/appearance', 'review/aroma', 'review/overall', 'review/palate', 'review/taste'], axis=1).values

In [12]:
X.shape

(37494, 896)

In [13]:
y = full_data['review/overall'].values

In [14]:
full_data.to_csv('final_data.csv')

In [18]:
full_data.head()

,beer/ABV,review/appearance,review/aroma,review/overall,review/palate,review/taste,isdst,"""The Wind Cried Mari..."" Scottish Heather Ale",1906 Reserva Especial,2X Chocolate Porter,...,90,91,92,93,94,95,96,97,98,99
0,5.0,4.0,4.0,4.0,4.0,4.0,0.0,0.0,0.0,0.0,...,-0.003817,-0.001904,-0.002556,0.003672,-0.000728,-0.002721,-0.003397,0.003552,0.000400,0.003669
1,11.0,4.0,3.5,3.5,3.5,3.0,0.0,0.0,0.0,0.0,...,0.008872,-0.008459,-0.005390,0.001952,0.003362,-0.000566,-0.001022,-0.000789,-0.004414,0.005061
2,4.7,3.5,4.0,3.5,3.5,3.5,0.0,0.0,0.0,0.0,...,-0.003155,-0.001823,0.002242,0.002276,-0.000769,0.002075,0.004495,-0.000685,-0.007607,0.003911
3,4.4,3.0,3.0,2.5,3.0,3.0,0.0,0.0,0.0,0.0,...,-0.001222,0.002758,-0.001472,0.000550,0.008042,0.012155,0.009179,0.004690,0.001785,0.003160
4,4.4,4.0,3.0,3.0,3.5,2.5,0.0,0.0,0.0,0.0,...,0.003644,-0.009948,-0.001942,0.005490,0.003194,0.000178,0.004289,0.001215,-0.002551,0.005537


In [19]:
full_data.dropna(inplace=True)

In [2]:
full_data[full_data.isnull()]

NameError: name 'full_data' is not defined

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [837]:
l = [Layer(250)] + ([Layer(20, ReLU())] * 10) + [Layer(1)]

In [849]:
network_1 = FullyConnectedNeuralNetwork(
    layers=l,
    loss = SquaredErrorLoss(),
    learning_rate= 0.00000000000000000001
)

In [850]:
n=10
for i in range(0, len(y_train), n):
    network_1.train(X_train[i:i+n], y_train[i:i+n])

In [852]:
network_1.feedforward([X_test[5]])

array([[-684.74625613]])

In [853]:
predicted = network_1.feedforward(X_test)

In [854]:
predicted

array([[-1774.98918098],
       [-2845.03400576],
       [-2480.15262322],
       ..., 
       [-1133.30312844],
       [-5720.87303684],
       [-7483.86534858]])

In [855]:
predicted.min(), predicted.max()

(-21600.749241491692, 121.08819412854741)

In [856]:
np.mean((y_test - predicted.flatten())**2)

13301956.922905698

In [23]:
X_train

array([[  4.40000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -1.57359374e-03,  -4.17524945e-03,   6.08118718e-03],
       [  1.12000000e+01,   0.00000000e+00,   0.00000000e+00, ...,
         -5.61188148e-03,   9.76825454e-03,   6.26696735e-03],
       [  5.90000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -1.32121412e-04,  -6.73356530e-03,   2.80870574e-03],
       ..., 
       [  4.40000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -3.83761954e-03,  -1.08731993e-02,   4.39019528e-03],
       [  1.05000000e+01,   0.00000000e+00,   0.00000000e+00, ...,
         -4.31864509e-03,  -5.39410540e-03,   4.79386121e-03],
       [  1.20000000e+01,   0.00000000e+00,   0.00000000e+00, ...,
          2.14395446e-03,  -2.11244398e-03,   4.37316158e-03]])

In [31]:
y_train = (y_train - np.mean(y_train))/(np.std(y_train))

In [33]:
X_train = (X_train - np.mean(X_train))/(np.std(X_train))

nan

In [16]:
X_train

array([[  6.60000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -5.43027851e-03,   4.53009133e-03,   3.44912530e-03],
       [  8.50000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -4.24265087e-04,  -1.75124991e-03,   3.06964660e-03],
       [  8.30000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -8.63554913e-03,   5.45475078e-03,   5.04183697e-03],
       ..., 
       [  6.30000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -6.23733596e-04,  -1.26662604e-03,   4.72117017e-03],
       [  1.20000000e+01,   0.00000000e+00,   0.00000000e+00, ...,
          3.20949235e-03,   3.91589245e-03,   5.28097794e-03],
       [  7.20000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -9.33489452e-04,  -1.09086857e-02,   5.34232776e-03]])

In [34]:
# specify the model architecture
# l = [layers.Dense(896, activation="relu")] + ([layers.Dense(20, activation="relu")] * 20) + [layers.Dense(1)]
model = tf.keras.Sequential([
    layers.Dense(896, activation="relu"),
    layers.Dense(1)
])

# specify the loss function and optimization function
model.compile(optimizer=tf.train.GradientDescentOptimizer(0.000000000000001),
              loss='mse')

# fit the model to data
model.fit(X_train, y_train, epochs=5, batch_size=128)

Epoch 1/5
28120/28120 [==============================] - 2s 55us/step - loss: nan
Epoch 2/5
28120/28120 [==============================] - 1s 51us/step - loss: nan
Epoch 3/5
28120/28120 [==============================] - 1s 46us/step - loss: nan
Epoch 4/5
28120/28120 [==============================] - 1s 45us/step - loss: nan
Epoch 5/5
28120/28120 [==============================] - 1s 45us/step - loss: nan


In [18]:
model.predict(X_test)

array([[ nan],
       [ nan],
       [ nan],
       ..., 
       [ nan],
       [ nan],
       [ nan]], dtype=float32)

In [56]:
network = FullyConnectedNeuralNetwork(
    layers=[Layer(10), Layer(20, ReLU()), Layer(1)],
    loss = SquaredErrorLoss(),
    learning_rate=0.001
)

In [57]:
diabetes = datasets.load_diabetes()

In [58]:
X = diabetes.data

In [59]:
y = diabetes.target

In [62]:
for i in range(len(y)):
    network.train([X[i]], y[i])

In [63]:
np.mean((y - network.feedforward(X))**2)

8344.7284204534008